In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.33:7077") \
        .appName("IdaAkerholm_1")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

        # RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 13:06:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sqlContext = SQLContext(spark_session.sparkContext)
sqlContext

/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
reddit_df = sqlContext.read.json("/home/ubuntu/data/corpus-webis-tldr-17.json")


In [4]:
reddit_df.printSchema()


root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [5]:
reddit_df.show(1)

+----------------+--------------------+--------------------+-----------+-------+--------------------+---------+------------+--------------------+-----------+-----+
|          author|                body|             content|content_len|     id|      normalizedBody|subreddit|subreddit_id|             summary|summary_len|title|
+----------------+--------------------+--------------------+-----------+-------+--------------------+---------+------------+--------------------+-----------+-----+
|raysofdarkmatter|I think it should...|I think it should...|        178|c69al3r|I think it should...|     math|    t5_2qh0n|Shifting seasonal...|          8| NULL|
+----------------+--------------------+--------------------+-----------+-------+--------------------+---------+------------+--------------------+-----------+-----+
only showing top 1 row



In [6]:
reddit_df.count()

3848330

In [7]:
#samp_df = reddit_df.sample(withReplacement=False, fraction=0.1)

In [8]:
#df = samp_df.cache()

In [9]:
#df.count()

In [10]:
df = reddit_df

In [11]:
df.rdd.getNumPartitions()

147

In [12]:
df2 = df.drop("content_len","id","normalizedBody","subreddit_id","summary_len")

In [13]:
df2.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- title: string (nullable = true)



In [14]:
#Count number of times each subreddit occur
df_subreddit_count = df2.groupBy("subreddit").count()
df_subreddit_count.show()

+--------------------+-----+
|           subreddit|count|
+--------------------+-----+
|               anime| 5868|
|          MensRights| 5136|
|              travel| 3076|
|londonfootballmeetup|    6|
|               HPMOR|  207|
|     youtubecomments|    8|
|        SaltLakeCity|  238|
| UnresolvedMysteries|  121|
|          MLBTheShow|  173|
|           metro2033|   16|
|        marvelheroes|  416|
|             DRKCoin|   26|
|              AdPorn|   17|
|          costa_rica|   31|
|          television| 1489|
|  Anarcho_Capitalism| 1400|
|    fatpeoplestories| 4926|
|       SanJoseSharks|  121|
|              Hawaii|  248|
|             wilfred|   63|
+--------------------+-----+
only showing top 20 rows



In [15]:
df_subreddit_count.count()

29651

In [16]:
df3= df_subreddit_count.filter(df_subreddit_count['subreddit']=='depression')
df3.show()

+----------+-----+
| subreddit|count|
+----------+-----+
|depression|11365|
+----------+-----+



In [17]:
df3= df_subreddit_count.filter(df_subreddit_count['subreddit']=='mentalhealth')
df3.show()

+------------+-----+
|   subreddit|count|
+------------+-----+
|mentalhealth|  623|
+------------+-----+



In [18]:
df3= df_subreddit_count.filter(df_subreddit_count['subreddit']=='Anxiety')
df3.show()

+---------+-----+
|subreddit|count|
+---------+-----+
|  Anxiety| 4126|
+---------+-----+

